<a href="https://colab.research.google.com/github/georgeaidinis/NTUA/blob/master/Voice%20Processing/Lab%201/Prelab/Prelab_voice_proc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Αϊδίνης Γιώργος  (el16031)

> Τσιλιβής Θοδωρής (el16032)



---
# 1η Εργαστηριακή Άσκηση - Μέρος πρώτο: Ορθογράφος

Πριν αρχίσουμε τα βήματα για την προετοιμασία του εργαστηρίου, πρέπει να εγκαταστήσουμε κάποια εργαλεία που θα χρησιμοποιήσουμε στην εργασία αυτή. Πρώτα είναι η βιβλιοθήκη OpenFST (1.6.1), για την οποία χρησιμοποιούμε το έτοιμο bash script.

In [0]:
!chmod +x install_openfst.sh
!./install_openfst.sh

Ακόμα, θα πρέπει να εγκαταστήσουμε τις βιβλιοθήκες του NLTK (Natural Language Toolkit) καθώς και του graphviz, που μας επιτρέπει να σχεδιάζουμε τα FST που δημιουργούμε.

In [0]:
!pip install nltk
!sudo apt install python-pydot python-pydot-ng graphviz

---
# Βήμα 1ο

Για το Βήμα αυτό, επιλέξαμε και τα δύο προτεινόμενα βιβλία από το Project Guternberg, για να έχουμε περισσότερα δεδομένα.

Το τελικό αρχείο, ονομάζεται corpora.txt . 

Ιδανικά, θα θέλαμε να είχαμε όσο το δυνατόν περισσοτερα δεδομενα, καθως αυτο θα μας επετρεπε να εχουμε ευρυτερο λεξιλογιο, δηλαδη να αναγνωριζει το συστημα μας περισσοτερες λεξεις. Ακομα, θα μπορουσαμε να εχουμε στην διαθεση μας διαφορετικες γραφες των ιδιων λεξεων, που ειναι χρησιμο σε εξειδικευμενους ορθογραφους.

Προσδιοριζουμε την κωδικοποιηση χαρακτηρων σε UTF-8, για να αναγνωρισθουν τυχον "περιεργοι" χαρακτηρες.

Ακομα, ειναι χρησιμο να κανουμε import τωρα ο,τι βιβλιοθηκες της python θα χρησιμοποιησουμε αργοτερα:

In [0]:
# -*- coding: utf-8 -*-


import itertools
from nltk.tokenize.regexp import regexp_tokenize

---
# Βήμα 2ο



> α)  Γραφουμε μια συναρτηση που την ονομαζουμε identity_preprocess: (αρκετα απλο)




In [0]:
"""Reads a string and returns it. Returns string."""
def identity_preprocess(stringaki):
	return stringaki




> β) Γραφουμε μια συναρτηση που να δεχεται δυο ορισματα, το ονομα αρχειου, και μια συναρτηση της επιλογης μας και να διαβαζει το αρχειο γραμμη - γραμμη καλωντας την συναρτηση που εχουμε δωσει σαν ορισμα σε καθε γραμμη:

In [0]:
"""Takes 2 arguments, one being the filepath and the second
being the name of the method that reads the string, defaulting
to the above method, 'identity_preprocess'. Returns a list of
strings, that are the whole text of the file."""
def read_from_file(filepath, myfunc = identity_preprocess):
	lines = []
	with open(filepath) as file:
		for line in file:
			lines.append(myfunc(line))
	return lines

> γ) Γραφουμε εδω 2 συναρτησεις, η μια ειναι η συναρτηση clean_text που "καθαριζει" το κειμενο που της δινουμε απο οποιον χαρακτηρα δεν ειναι γραμμα, και στην συνεχεια, την tokenize, που παιρνει ενα τετοιο καθαρο κειμενο και το χωριζει σε μερη:

In [0]:
"""Takes one string and returns a string that has only lowercase
letters, that means exluding any non-alphanumeric sumbols, numbers
punctuation."""
def clean_text(text):
	#replace new line and carriage return
	text = text.replace("\n", " ").replace("\r", " ")
	#replace the numbers, symbols and punctuation with space
	punc_list = '~`!@#$£½%^&*()_+-—“”’‘\'=}{|:à"?>æ<âç,œ./è;é][}' + '1234567890'
	t = str.maketrans(dict.fromkeys(punc_list, " "))
	text = text.translate(t)
	#make everything lowercase
	text = text.lower()
	return text



"""Takes a text and splits it between whitespaces or \n. Returns
a list of strings"""
def tokenize(text):
	text = clean_text(text)
	text = text.split()
	return text




> δ) μια καλη συναρτηση που βρηκαμε οτι εχει καλα αποτελεσματα ειναι η εξης:



In [0]:

"""Uses the imported library from nltk to return a list of strings
created by using regular expressions. It is one of the fastest 
tokenizers."""
def Tokenize(text):
	#text = clean_text(text)
	return regexp_tokenize(text, pattern = '\s+', gaps = True)


Στις προσπαθειες μας, βρηκαμε οτι η ετοιμη συναρτηση ενω ειναι παρα πολυ γρηγορη, (πολυ πιο γρηγορη απο την δικη μας, ακομα και για τον σχετικα μικρο αριθμο δεδομενων μας), επιτρεπει λιγο λιγοτερες λεξεις (~170k vs ~173k) να "περασουν" ομως καταληγουμε με πολυ μεγαλο λεξιλογιο (κατι που θα φανει μετεπειτα): (παραθετουμε και την συναρτηση flatten, που αν και δεν ζητειται, ειναι χρησιμη καθως μετατρεπει μια λιστα λιστων σε μια λιστα)

In [14]:
"""Takes a list of lists and returns a list. Elements are kept in
order."""
def flatten(l):
	return list(itertools.chain(*l))
 

lines = read_from_file("corpora.txt")
text = []
text1 = []
for line in lines:
    text.append(Tokenize(line))
    text1.append(tokenize(line))
text = flatten(text)
text1 = flatten(text1)
print(len(text))
print(len(text1))

170792
173206


---
# Βήμα 3ο



> α)  Για να βρουμε το λεξιλογιο μας, φτιαχνουμε την συναρτηση unique_tokens, που παιρνει μια λιστα απο strings και αφαιρει τις πολλαπλες εμφανισεις:

In [0]:
"""Takes a list of strings (tokens) and reduces it to only unique
elements. (gets rid of the duplicates). Returns a list of strings
that are unique and sorted."""
def unique_tokens(tokens):
	lexicon = list(set(tokens))
	lexicon.sort()
	return lexicon

> β)  Για να βρουμε το αλφαβητο του corpus, φτιαχνουμε την συναρτηση alphabet, που παιρνει μια λιστα απο strings επιστρεφει μια λιστα απο χαρακτηρες, χωρις πολλαπλες εμφανισεις:

Εκτελωντας τις παρακατω εντολες, μπορουμε να δουμε οτι τα λεξιλογια που δημιουργουνται απο τις διαφορετικες μεθοδους tokenization ειναι κατα πολυ διαφορετικα:

In [20]:
lines = read_from_file("corpora.txt")
text = []
for line in lines:
    text.append(Tokenize(line))
text = flatten(text)
Lexicon = unique_tokens(text)
print("NLTK tokenize(): ",len(Lexicon))
text = []
for line in lines:
    text.append(tokenize(line))
text = flatten(text)
Lexicon = unique_tokens(text)
print("Our tokenize(): ",len(Lexicon))

NLTK tokenize():  21672
Our tokenize():  11249


In [0]:
"""Takes a string and returns a list of the characters 
that were used to make the strings."""
def split_word(word):
	return [char for char in word]



"""Takes a list of strings as an input and returns a list of chars.
The characters that are returned are used to make the elements of 
the input strings, but they are returned duplicate-free and sorted."""
def alphabet(words):
	chars = []
	for word in words:
		chars.append(split_word(word))
	alpbt = list(set(flatten(chars)))
	alpbt.sort()
	return alpbt

---
# Βήμα 4ο


Για να δημιουργησουμε το αρχειο chars.syms που μας χρειαζεται για μετα, φιταχνουμε δυο συναρτησεις, μια για να κανει το (προηγουμενως δημιουργημενο) αλφαβητο indexed, δηλαδη καθε χαρακτηρας να αντιστοιχιζεται σε εναν φυσικο αριθμο, και το ε ('<epsilon>') να αντιστοιχιζεται στο 0. Ακομα, φτιαχνουμε μια συναρτηση που να παιρνει ενα τετοιο στοιχισμενο αλφαβητο και να το αποθηκευει σε ενα αρχειο της επιλογης μας (προεπιλεγμενο ονομα ειναι το "chars.syms"):


In [0]:
"""Takes an alphabet (list of chars) and assigns each character 
an index. Index 0 is <epsilon> (ε). Returns a list that has n+1 
tuples, each tuple being the character with its assigned index."""
def alphabet_indexing(alphabet):
	indexed_alphabet = []
	indexed_alphabet.append(("<epsilon>", 0))
	i = 1;
	for letter in alphabet:
		indexed_alphabet.append((letter, str(i)))
		i+=1
	return indexed_alphabet



"""Takes two arguments. The first is an  alphabet and the 
second is the name of the file to write the output. The 
method takes the indexed alphabet and writes it to a file 
with the name given"""
def symbols_file(alphabet, filename = "chars.syms"):
	f = open(filename, "w+")
	for pair in alphabet:
		f.write(pair[0] + "\t\t" + str(pair[1]) + "\n")


---
# Βήμα 5ο


   α) Για to βημα 5ο, αρχικα υποθεσαμε οτι πρεπει να υλοποιησουμε εκτος των αλλων και την συναρτηση για την αποσταση Levenshtein, οποτε αν και (μετα απο ξεκαθαρισμα) δεν ειναι απαραιτητο, την παραθετουμε εδω για κριτικη, καθως και για να υπαρχει σε τυχον περιπτωση που την χρειαστουμε:

In [0]:
"""This function computes the Levenshtein distance between two 
chars s,t. It uses the algorithm mentioned in the instructions
of the lab, and is horribly inefficient. Takes two arguments, both
chars and returns an integer that is the levenshtein distance of
the two strings."""
def my_levenshtein(s,t):
	#same characters, no edit => Levenshtein cost 0
	if (s==t):
		return 0
	#<epsilon> with some other character or any character with <epsilon>
	#either insertion or deletion => Levenshtein cost 1
	elif (s=='<epsilon' and t!='<epsilon>') or (s!='<epsilon' and t=='<epsilon>'):
		return 1
	#two different chars, substiturion => Levenshtein cost 1
	else:
		return 1



"""This function computes the Levenshtein distance between two 
strings s,t. It uses the iterative algorithm, that creates a 
2d matrix of the characters of the strings, along with the 
values it needs to compute the sum of the weights of the insert
/delete/substitute logic. It is a much faster algorithm than the
recursive one. The method takes 3 arguments, string s, string t
and a tuple of 3 integeres that are the cost of deletion, insertion
and substitution respectively. Defaults to (1,1,1) (equal cost).
I have taken the code for this method from:
https://www.python-course.eu/levenshtein_distance.php 
Another method could have been the method given in
the python package python-Levenshtein (0.12.0) that uses python to 
instruct C to compute the afforementioned array, with much faster 
execution times."""
def iterative_levenshtein(s, t, costs=(1, 1, 1)):
    """ 
        iterative_levenshtein(s, t) -> ldist
        ldist is the Levenshtein distance between the strings 
        s and t.
        For all i and j, dist[i,j] will contain the Levenshtein 
        distance between the first i characters of s and the 
        first j characters of t
        
        costs: a tuple or a list with three integers (d, i, s)
               where d defines the costs for a deletion
                     i defines the costs for an insertion and
                     s defines the costs for a substitution
    """
    rows = len(s)+1
    cols = len(t)+1
    deletes, inserts, substitutes = costs
    
    dist = [[0 for x in range(cols)] for x in range(rows)]
    # source prefixes can be transformed into empty strings 
    # by deletions:
    for row in range(1, rows):
        dist[row][0] = row * deletes
    # target prefixes can be created from an empty source string
    # by inserting the characters
    for col in range(1, cols):
        dist[0][col] = col * inserts
        
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0
            else:
                cost = substitutes
            dist[row][col] = min(dist[row-1][col] + deletes,
                                 dist[row][col-1] + inserts,
                                 dist[row-1][col-1] + cost) # substitution
    """for r in range(rows):
        print(dist[r])
    """ 
    return dist[row][col]

Προφανως η ασκηση δεν ανεφερε αυτο, οποτε για την δημιουργια του μετατροπεα μιας καταστασης, αρχικα δημιουργουμε τα αρχεια που χρειαζομαστε, (τα αρχεια για τα συμβολα ειναι ηδη ετοιμα, καθως ειναι ομοια με τα αρχεια του στοιχισμενου αλφαβητου που περιεχεται στο αρχειο chars.syms. Για τον σκοπο αυτο, φτιαξαμε μια μεθοδο για να τυπωνει τα περιεχομενα του fst τοσο στο stdout, οσο και στο αρχειο με ονομα "text.txt" που θα χρησιμοποιησουμε αργοτερα. O ακριβης αριθμος των ακμων στο fst ειναι (πληθος αλφαβητου)^2 -1, καθως θελουμε καθε γραμμα να εχει καποια ενεργεια με οποιοδηποτε αλλο γραμμα του αλφαβητου, εκτος απο το ε, που με τον εαυτο του δεν θελουμε να εχει. 

In [0]:
def text_fst(Alphabet, filename = "text.txt", print_lines = False):
    f = open(filename, "w+")
    if (print_lines):
        for pair in Alphabet:
            for pair1 in Alphabet:
                if  (pair[0] == pair1[0] =="<epsilon>"):
                    continue 
                elif (pair[0] == pair1[0] and pair[0]!="<epsilon>"):
                    print("0" + " 0 " + pair[0] + " " + pair1[0] + " 0")
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 0" + "\n")
                elif (pair[0]=='<epsilon>'):
                    print("0" + " 0 " + pair[0] + " " + pair1[0] + " 1")
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 1"+ "\n")
                elif (pair1[0]=='<epsilon>'):
                    print("0" + " 0 " + pair[0] + " " + pair1[0] + " 1")
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 1"+ "\n")
                else:
                    print("0" + " 0 " + pair[0] + " " + pair1[0] + " 1")
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 1"+ "\n")
        print("0")
        f.write("0")
    else:
        for pair in Alphabet:
            for pair1 in Alphabet:
                if  (pair[0] == pair1[0] =="<epsilon>"):
                    continue 
                elif (pair[0] == pair1[0] and pair[0]!="<epsilon>"):
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 0" + "\n")
                elif (pair[0]=='<epsilon>'):
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 1"+ "\n")
                elif (pair1[0]=='<epsilon>'):
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 1"+ "\n")
                else:
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 1"+ "\n")
        f.write("0")
    f.close()


Για να δημιουργησουμε τα αρχεια, θα πρεπει να εκτελεσουμε τον παρακατω κωδικα:

In [0]:
lines = read_from_file("corpora.txt")
#print(len(lines))
text = []
for line in lines:
    text.append(tokenize(line))
text = flatten(text)
#print(len(text))
Lexicon = unique_tokens(text)
#print(len(Lexicon))
Alphabet = alphabet(Lexicon)
#print(len(Alphabet))
Alphabet = alphabet_indexing(Alphabet)
#print(Alphabet)
symbols_file(Alphabet)
text_fst(Alphabet)

Στην συνεχεια, για να δημιουργησουμε το fst, θα πρεπει να εκτελεστει το παρακτω shell script:

In [0]:
!fstcompile -isymbols=chars.syms -osymbols=chars.syms text.txt > fst.fst
!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait fst.fst  | dot -Tjpg >fst.jpg
!fstinfo fst.fst

Η πρωτη εντολη κανει compile τις πληροφοριες του αρχειο που δημιουργησαμε προηγουμενως για το one state fst, και παιρνει σαν input και output symbols τους χαρακτηρες του chars.syms (που ειναι και η αλφαβητα μας).
Η δευτερη εντολη σχεδιαζει το fst που δημιουργηθηκε με τις βιβλιοθηκες dot και ghraphviz, και το αποθηκευει στο αρχειο fst.jpg. Η τριτη εντολη ειναι για να δουμε οτι ολα πηγαν καλα, δηλαδη οτι δημιουργηθηκε και εχει τα καταλληλα χαρακτηρηστικα. Επειδη ο μετατροπεας μιας καταστασης για την περιπτωση μας ειναι εξαιρετικα μεγαλος και η εικονα (jpg) δεν μας δειχνει καθαρα το τι συμβαινει, θα κανουμε μια μικροτερη αποτυπωση, απλα και μονο για λογους κατανοησης της λειτουργιας του κωδικα:

In [0]:
Alphabet = [('<epsilon>', 0), ('a', '1'), ('b', '2'), ('c', '3'), ('d', '4')]
symbols_file(Alphabet, "chars1.syms")
text_fst(Alphabet, "text1.txt", False)

Τα αρχεια αυτα εχουν 25 ακμες, οποτε ειναι αρκετα πιο ευκολο να διακρινουμε τι γινεται απο την εικονα:

In [0]:
!fstcompile -isymbols=chars1.syms -osymbols=chars1.syms text1.txt > fst1.fst
!fstdraw --isymbols=chars1.syms --osymbols=chars1.syms -portrait fst1.fst  | dot -Tjpg >fst1.jpg
!fstinfo fst1.fst

Ο μετατροπεας εισοδου αν παρουμε το shortest path, αυτο σημαινει οτι δεν επιφερει καμια αλλαγη, δηλαδη η λεξη που παιρνει στην εισοδο και η λεξη που παιρνουμε στην εξοδο ειναι ιδιες.

   β) Ιδανικα, αντι να εχουμε κανονες για τα βαρη, θα επρεπε αν ειχαμε πολυ περισσοτερα δεδομενα να εφαρμοζουμε πιθανοτικες μεθοδους για να προσδιορισουμε τα βαρη καλυτερα στο μοντελο μας.

---
# Βήμα 6ο
